In [2]:
import cv2
import mediapipe as mp
import numpy as np
import joblib

# Load the new Random Forest model trained on hand landmarks
model = joblib.load('hand_landmark_random_forest_model.pkl')

# Initialize Mediapipe Hands model (simplified import)
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# Function to preprocess hand landmarks for prediction
def preprocess_landmarks(landmarks):
    return np.array([[lm.x, lm.y, lm.z] for lm in landmarks]).flatten()

# Function to predict hand pose based on hand landmarks
def predict_hand_pose(landmarks):
    processed_landmarks = preprocess_landmarks(landmarks)
    prediction = model.predict([processed_landmarks])
    return prediction[0]

# Initialize webcam for video capture
cap = cv2.VideoCapture(1)

# Class mapping
class_map = {0: 'Start', 1: 'Pause', 2: 'Stop'}

# Start Mediapipe Hands model
with mp_hands.Hands(max_num_hands=2, min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Convert frame to RGB for Mediapipe
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Detect hands and get hand landmarks
        result = hands.process(rgb_frame)

        if result.multi_hand_landmarks:
            for hand_landmarks in result.multi_hand_landmarks:
                # Draw hand landmarks on the frame
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                # Predict hand pose based on landmarks
                predicted_class = predict_hand_pose(hand_landmarks.landmark)
                hand_pose = class_map[predicted_class]

                # Display the hand pose prediction
                cv2.putText(frame, f'Hand Pose: {hand_pose}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Show the frame with landmarks and prediction
        cv2.imshow('Hand Pose Detection', frame)

        # Exit on 'q' key press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Release the webcam and close the window
cap.release()
cv2.destroyAllWindows()


C:\Users\Owen Brown\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\Users\Owen Brown\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\Users\Owen Brown\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\Users\Owen Brown\anaconda3\Lib\

In [8]:
import cv2
import mediapipe as mp
import numpy as np
import joblib
from collections import deque

# Load the new Random Forest model trained on hand landmarks
model = joblib.load('hand_landmark_random_forest_model.pkl')

# Initialize Mediapipe Hands model
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# Function to preprocess hand landmarks for prediction
def preprocess_landmarks(landmarks):
    return np.array([[lm.x, lm.y, lm.z] for lm in landmarks]).flatten()

# Function to predict hand pose based on hand landmarks
def predict_hand_pose(landmarks):
    processed_landmarks = preprocess_landmarks(landmarks)
    prediction = model.predict([processed_landmarks])
    return prediction[0]

# Initialize webcam for video capture
cap = cv2.VideoCapture(1)

# Class mapping
class_map = {0: 'Start', 1: 'Pause', 2: 'Stop'}

# Store the last 3 predictions
last_predictions = deque(maxlen=3)

# Start Mediapipe Hands model
with mp_hands.Hands(max_num_hands=2, min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Convert frame to RGB for Mediapipe
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Detect hands and get hand landmarks
        result = hands.process(rgb_frame)

        if result.multi_hand_landmarks:
            for hand_landmarks in result.multi_hand_landmarks:
                # Draw hand landmarks on the frame
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                # Predict hand pose based on landmarks
                predicted_class = predict_hand_pose(hand_landmarks.landmark)
                hand_pose = class_map[predicted_class]

                # Add the prediction to the deque (stores last 3 predictions)
                last_predictions.append(hand_pose)

                # Get bounding box coordinates around the hand
                h, w, _ = frame.shape
                x_min, x_max, y_min, y_max = w, 0, h, 0
                for lm in hand_landmarks.landmark:
                    x, y = int(lm.x * w), int(lm.y * h)
                    x_min, x_max = min(x, x_min), max(x, x_max)
                    y_min, y_max = min(y, y_min), max(y, y_max)

                # Draw bounding box around the hand
                cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)

                # Display the predicted hand pose near the bounding box
                cv2.putText(frame, f'Hand Pose: {hand_pose}', (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

        # Show the last 3 classifications on the left side of the screen
        for i, pred in enumerate(last_predictions):
            cv2.putText(frame, f'{i+1}: {pred}', (10, 50 + i * 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # Show the frame with landmarks, bounding box, and predictions
        cv2.imshow('Hand Pose Detection', frame)

        # Exit on 'q' key press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Release the webcam and close the window
cap.release()
cv2.destroyAllWindows()


C:\Users\Owen Brown\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\Users\Owen Brown\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\Users\Owen Brown\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\Users\Owen Brown\anaconda3\Lib\

In [18]:
import cv2
import mediapipe as mp
import numpy as np
import joblib
import time
from collections import deque

# Load the new Random Forest model trained on hand landmarks
model = joblib.load('hand_landmark_random_forest_model.pkl')

# Initialize Mediapipe Hands model
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# Function to preprocess hand landmarks for prediction
def preprocess_landmarks(landmarks):
    return np.array([[lm.x, lm.y, lm.z] for lm in landmarks]).flatten()

# Function to predict hand pose based on hand landmarks
def predict_hand_pose(landmarks):
    processed_landmarks = preprocess_landmarks(landmarks)
    prediction = model.predict([processed_landmarks])
    return prediction[0]

# Initialize webcam for video capture
cap = cv2.VideoCapture(1)

# Class mapping
class_map = {0: 'Start Particle Counter', 1: 'Pause Particle Counter', 2: 'Stop Particle Counter'}

# Store the last 3 predictions
last_predictions = deque(maxlen=3)

# Variables to track consistent detection
current_class = None
class_start_time = None
consistency_duration = 3  # seconds to wait before displaying the new class

# Start Mediapipe Hands model
with mp_hands.Hands(max_num_hands=2, min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Convert frame to RGB for Mediapipe
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Detect hands and get hand landmarks
        result = hands.process(rgb_frame)

        detected_class = None  # Reset detected class in each frame

        if result.multi_hand_landmarks:
            for hand_landmarks in result.multi_hand_landmarks:
                # Draw hand landmarks on the frame
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                # Predict hand pose based on landmarks
                predicted_class = predict_hand_pose(hand_landmarks.landmark)
                detected_class = class_map[predicted_class]

                # Get bounding box coordinates around the hand
                h, w, _ = frame.shape
                x_min, x_max, y_min, y_max = w, 0, h, 0
                for lm in hand_landmarks.landmark:
                    x, y = int(lm.x * w), int(lm.y * h)
                    x_min, x_max = min(x, x_min), max(x, x_max)
                    y_min, y_max = min(y, y_min), max(y, y_max)

                # Draw bounding box around the hand
                cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)

        # Check if the detected class is consistent for 10 seconds
        if detected_class:
            if detected_class == current_class:
                # If the same class is still detected, check the elapsed time
                elapsed_time = time.time() - class_start_time
                if elapsed_time >= consistency_duration:
                    # Once 10 seconds have passed, add the class to the last predictions
                    if len(last_predictions) == 0 or last_predictions[-1] != current_class:
                        last_predictions.append(current_class)
            else:
                # Detected a new class, reset timer
                current_class = detected_class
                class_start_time = time.time()

        # Show the last 3 classifications on the left side of the screen
        for i, pred in enumerate(last_predictions):
            cv2.putText(frame, f'{i+1}: {pred}', (10, 50 + i * 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # Show the frame with landmarks, bounding box, and predictions
        cv2.imshow('Hand Pose Detection', frame)

        # Exit on 'q' key press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Release the webcam and close the window
cap.release()
cv2.destroyAllWindows()


C:\Users\Owen Brown\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\Users\Owen Brown\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\Users\Owen Brown\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\Users\Owen Brown\anaconda3\Lib\

In [1]:
import streamlit as st
import cv2
import mediapipe as mp
import numpy as np
import joblib
import pandas as pd
from collections import deque
from datetime import datetime

# Load the Random Forest model trained on hand landmarks
model = joblib.load('hand_landmark_random_forest_model.pkl')

# Initialize Mediapipe Hands model
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# Function to preprocess hand landmarks for prediction
def preprocess_landmarks(landmarks):
    return np.array([[lm.x, lm.y, lm.z] for lm in landmarks]).flatten()

# Function to predict hand pose based on hand landmarks
def predict_hand_pose(landmarks):
    processed_landmarks = preprocess_landmarks(landmarks)
    prediction = model.predict([processed_landmarks])
    return prediction[0]

# Logging predictions in a list
logged_predictions = []

# Class mapping
class_map = {0: 'Start', 1: 'Pause', 2: 'Stop'}

# Variables to track consistent detection
current_class = None
class_start_time = None
consistency_duration = 10  # seconds to wait before displaying the new class
last_predictions = deque(maxlen=3)

# Streamlit app layout
st.title("Hand Pose Detection")
st.image("img_pldrn.png", use_column_width=True)  # Background image

# Define the video capture window layout and buttons
start_button = st.button("Start")
stop_button = st.button("Stop")
print_button = st.button("Print Results")

# Video capture placeholder
video_placeholder = st.empty()

# Initialize variables
cap = None
running = False

# Start video capture when Start button is clicked
if start_button:
    running = True
    cap = cv2.VideoCapture(0)

# Stop video capture when Stop button is clicked
if stop_button and cap:
    cap.release()
    running = False

# Capture and display video in real-time
if running and cap is not None:
    with mp_hands.Hands(max_num_hands=2, min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            # Convert frame to RGB for Mediapipe
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # Detect hands and get hand landmarks
            result = hands.process(rgb_frame)

            detected_class = None

            if result.multi_hand_landmarks:
                for hand_landmarks in result.multi_hand_landmarks:
                    # Draw hand landmarks on the frame
                    mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                    # Predict hand pose based on landmarks
                    predicted_class = predict_hand_pose(hand_landmarks.landmark)
                    detected_class = class_map[predicted_class]

                    # Get bounding box coordinates around the hand
                    h, w, _ = frame.shape
                    x_min, x_max, y_min, y_max = w, 0, h, 0
                    for lm in hand_landmarks.landmark:
                        x, y = int(lm.x * w), int(lm.y * h)
                        x_min, x_max = min(x, x_min), max(x, x_max)
                        y_min, y_max = min(y, y_min), max(y, y_max)

                    # Draw bounding box around the hand
                    cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)

            # Check if the detected class is consistent for 10 seconds
            if detected_class:
                if detected_class == current_class:
                    elapsed_time = time.time() - class_start_time
                    if elapsed_time >= consistency_duration:
                        if len(last_predictions) == 0 or last_predictions[-1] != current_class:
                            last_predictions.append(current_class)
                            logged_predictions.append({
                                "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                                "prediction": current_class
                            })
                else:
                    current_class = detected_class
                    class_start_time = time.time()

            # Display the frame with landmarks and bounding box
            video_placeholder.image(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB), channels="RGB")

# Print results to a spreadsheet when Print button is clicked
if print_button and logged_predictions:
    df = pd.DataFrame(logged_predictions)
    file_name = f'logged_predictions_{datetime.now().strftime("%Y-%m-%d_%H-%M-%S")}.csv'
    df.to_csv(file_name, index=False)
    st.success(f"Results saved as {file_name}")
    st.dataframe(df)


2024-10-03 15:15:03.211 
  command:

    streamlit run C:\Users\Owen Brown\anaconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


In [11]:
import streamlit as st
import cv2
import mediapipe as mp
import numpy as np
import joblib
import pandas as pd
from collections import deque
from datetime import datetime
import time

# Load the Random Forest model trained on hand landmarks
model = joblib.load('hand_landmark_random_forest_model.pkl')

# Initialize Mediapipe Hands model
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# Function to preprocess hand landmarks for prediction
def preprocess_landmarks(landmarks):
    return np.array([[lm.x, lm.y, lm.z] for lm in landmarks]).flatten()

# Function to predict hand pose based on hand landmarks
def predict_hand_pose(landmarks):
    processed_landmarks = preprocess_landmarks(landmarks)
    prediction = model.predict([processed_landmarks])
    return prediction[0]

# Logging predictions in a list
logged_predictions = []

# Class mapping
class_map = {0: 'Start', 1: 'Pause', 2: 'Stop'}

# Variables to track consistent detection
current_class = None
class_start_time = None
consistency_duration = 10  # seconds to wait before displaying the new class
last_predictions = deque(maxlen=3)

# Streamlit app layout
st.title("Hand Pose Detection")
st.image("img_pldrn.png", use_column_width=True)  # Background image

# Define the video capture window layout and buttons
start_button = st.button("Start")
stop_button = st.button("Stop")
print_button = st.button("Print Results")

# Video capture placeholder
video_placeholder = st.empty()

# Initialize variables
cap = None
running = False

# Start video capture when Start button is clicked
if start_button:
    running = True
    cap = cv2.VideoCapture(0)

# Stop video capture when Stop button is clicked
if stop_button and cap:
    cap.release()
    running = False

# Capture and display video in real-time
if running and cap is not None:
    with mp_hands.Hands(max_num_hands=2, min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            # Convert frame to RGB for Mediapipe
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # Detect hands and get hand landmarks
            result = hands.process(rgb_frame)

            detected_class = None

            if result.multi_hand_landmarks:
                for hand_landmarks in result.multi_hand_landmarks:
                    # Draw hand landmarks on the frame
                    mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                    # Predict hand pose based on landmarks
                    predicted_class = predict_hand_pose(hand_landmarks.landmark)
                    detected_class = class_map[predicted_class]

                    # Get bounding box coordinates around the hand
                    h, w, _ = frame.shape
                    x_min, x_max, y_min, y_max = w, 0, h, 0
                    for lm in hand_landmarks.landmark:
                        x, y = int(lm.x * w), int(lm.y * h)
                        x_min, x_max = min(x, x_min), max(x, x_max)
                        y_min, y_max = min(y, y_min), max(y, y_max)

                    # Draw bounding box around the hand
                    cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)

            # Check if the detected class is consistent for 10 seconds
            if detected_class:
                if detected_class == current_class:
                    elapsed_time = time.time() - class_start_time
                    if elapsed_time >= consistency_duration:
                        if len(last_predictions) == 0 or last_predictions[-1] != current_class:
                            last_predictions.append(current_class)
                            logged_predictions.append({
                                "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                                "prediction": current_class
                            })
                else:
                    current_class = detected_class
                    class_start_time = time.time()

            # Display the frame with landmarks and bounding box
            video_placeholder.image(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB), channels="RGB")

# Print results to a spreadsheet when Print button is clicked
if print_button and logged_predictions:
    df = pd.DataFrame(logged_predictions)
    file_name = f'logged_predictions_{datetime.now().strftime("%Y-%m-%d_%H-%M-%S")}.csv'
    df.to_csv(file_name, index=False)
    st.success(f"Results saved as {file_name}")
    st.dataframe(df)


In [3]:
pwd

'C:\\Users\\Owen Brown\\Desktop\\face_detection\\Touch_Free\\Touch_Free'

In [15]:
model = joblib.load('hand_landmark_random_forest_model.pkl')


In [17]:
st.image("img_pldrn.png", use_column_width=True)


DeltaGenerator()